# Libraries + Data

In [60]:
# Importing necessary libraries

import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from joblib import dump, load

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


from sklearn.metrics import precision_score, f1_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score

In [15]:
# Data
# Running this in Google Colab; adjust if necessary.
df = pd.read_table("/content/train.tsv")
test_df = pd.read_table("/content/dev.tsv")

In [16]:
test_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,201359,303345,303346,Why are African-Americans so beautiful?,Why are hispanics so beautiful?,0
1,263843,69383,380476,I want to pursue PhD in Computer Science about...,I handle social media for a non-profit. Should...,0
2,172974,266948,175089,Is there a reason why we should travel alone?,What are some reasons to travel alone?,1
3,15329,29298,29299,Why are people so obsessed with having a girlf...,How can a single male have a child?,0
4,209794,314169,314170,What are some good baby girl names starting wi...,What are some good baby girl names starting wi...,0


# Training

## Data Cleaning

In [17]:
# Checking for Class Imbalance
dup_count_true = len(df[df['is_duplicate'] == 1])
dup_count_false = len(df[df['is_duplicate'] == 0])
total = len(df)

print(f"% of duplicates: {dup_count_true/total *100}%")
print(f"% of non-duplicates: {dup_count_false/total *100}%")



% of duplicates: 36.956496185628815%
% of non-duplicates: 63.04311199401299%


Training set is imbalanced, with a higher proportion of non-duplicate question pairs.

In [18]:
df.info()
print("Count")
df.isna().sum()
# no missing values in data; will skip checking for it

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255219 entries, 0 to 255218
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            255219 non-null  int64  
 1   qid1          255219 non-null  int64  
 2   qid2          255219 non-null  int64  
 3   question1     255218 non-null  object 
 4   question2     255218 non-null  object 
 5   is_duplicate  255218 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 11.7+ MB
Count


id              0
qid1            0
qid2            0
question1       1
question2       1
is_duplicate    1
dtype: int64

Training set contains 1 missing row.

In [31]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255218 entries, 0 to 255217
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            255218 non-null  int64  
 1   qid1          255218 non-null  int64  
 2   qid2          255218 non-null  int64  
 3   question1     255218 non-null  object 
 4   question2     255218 non-null  object 
 5   is_duplicate  255218 non-null  float64
 6   q1_processed  255218 non-null  object 
 7   q2_processed  255218 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 17.5+ MB


## Data Pre-processing

In [32]:
def preprocessing(question):
  # Remove capitalization
  question = str(question).lower().strip()

  # Remove Stop Words
  # From: https://stackoverflow.com/a/19794953

  contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
  }

  q_decontracted = []

  for word in question.split():
      if word in contractions:
          word = contractions[word]

      q_decontracted.append(word)

  question = ' '.join(q_decontracted)
  question = question.replace("'ve", " have")
  question = question.replace("n't", " not")
  question = question.replace("'re", " are")
  question = question.replace("'ll", " will")

  # Remove punctuation
  pattern = re.compile('\W')
  question = re.sub(pattern, ' ', question).strip()

  # Remove stopwords?

  return question

In [33]:
df['q1_processed'] = df['question1'].apply(preprocessing)
df['q2_processed'] = df['question2'].apply(preprocessing)

In [34]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_processed,q2_processed
0,133273,213221,213222,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0,how is the life of a math student could you d...,which level of prepration is enough for the ex...
1,402555,536040,536041,How do I control my horny emotions?,How do you control your horniness?,1.0,how do i control my horny emotions,how do you control your horniness
2,360472,364011,490273,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0,what causes stool color to change to yellow,what can cause stool to come out as little balls
3,150662,155721,7256,What can one do after MBBS?,What do i do after my MBBS ?,1.0,what can one do after mbbs,what do i do after my mbbs
4,183004,279958,279959,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0,where can i find a power outlet for my laptop ...,would a second airport in sydney australia be...


## Feature Extraction

Basic Features

In [36]:
df['q1_no_words'] = df['question1'].apply(lambda x : len(x.split(" ")))
df['q2_no_words'] = df['question2'].apply(lambda x : len(x.split(" ")))
df['sum_of_words'] = df['q1_no_words'] + df['q2_no_words']

df['q1_length'] = df['question1'].apply(len)
df['q2_length'] = df['question2'].apply(len)

df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_processed,q2_processed,q1_no_words,q2_no_words,sum_of_words,q1_length,q2_length
0,133273,213221,213222,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0,how is the life of a math student could you d...,which level of prepration is enough for the ex...,14,10,24,75,55
1,402555,536040,536041,How do I control my horny emotions?,How do you control your horniness?,1.0,how do i control my horny emotions,how do you control your horniness,7,6,13,35,34
2,360472,364011,490273,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0,what causes stool color to change to yellow,what can cause stool to come out as little balls,8,10,18,44,49
3,150662,155721,7256,What can one do after MBBS?,What do i do after my MBBS ?,1.0,what can one do after mbbs,what do i do after my mbbs,6,8,14,27,28
4,183004,279958,279959,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0,where can i find a power outlet for my laptop ...,would a second airport in sydney australia be...,13,20,33,67,121


Using TF-IDF and and cosine similarity to obtain a similarity score.

In [37]:
# TF-IDF:
vectorizer = TfidfVectorizer()
corpus = list(df['q1_processed']) + list(df['q2_processed'])

questions_vectorized = vectorizer.fit_transform(corpus)

print("n_samples: %d, n_features: %d" % questions_vectorized.shape)
print(questions_vectorized.shape)

q1_vectors = questions_vectorized[:(questions_vectorized.shape[0]//2), :]
q2_vectors = questions_vectorized[(questions_vectorized.shape[0]//2):, :]

n_samples: 510436, n_features: 70552
(510436, 70552)


In [44]:
%%time

vect = cosine_similarity(q1_vectors[0,:], q2_vectors[0,:]).flatten()

for row in range(1, q1_vectors.shape[0]):
  value = cosine_similarity(q1_vectors[row, :], q2_vectors[row, :]).flatten()
  vect = np.hstack((vect, value))


CPU times: user 5min 3s, sys: 751 ms, total: 5min 3s
Wall time: 5min 5s


In [40]:
print(len(vect))

255218


In [41]:
df['cosine_similarity'] = vect
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_processed,q2_processed,q1_no_words,q2_no_words,sum_of_words,q1_length,q2_length,cosine_similarity
0,133273,213221,213222,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0,how is the life of a math student could you d...,which level of prepration is enough for the ex...,14,10,24,75,55,0.036313
1,402555,536040,536041,How do I control my horny emotions?,How do you control your horniness?,1.0,how do i control my horny emotions,how do you control your horniness,7,6,13,35,34,0.238342
2,360472,364011,490273,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0,what causes stool color to change to yellow,what can cause stool to come out as little balls,8,10,18,44,49,0.336992
3,150662,155721,7256,What can one do after MBBS?,What do i do after my MBBS ?,1.0,what can one do after mbbs,what do i do after my mbbs,6,8,14,27,28,0.813123
4,183004,279958,279959,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0,where can i find a power outlet for my laptop ...,would a second airport in sydney australia be...,13,20,33,67,121,0.237551


## Prepping the DataFrame

In [42]:
train_X = df.copy()
train_X.drop(["id","qid1","qid2","question1","question2", "is_duplicate", "q1_processed", "q2_processed"], axis = 1, inplace = True)

train_Y = df["is_duplicate"]
print(train_X.shape)
print(train_Y.shape)

(255218, 6)
(255218,)


# Test Set

## Data Cleaning

In [46]:
test_df.info()
print("Count")
test_df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40430 entries, 0 to 40429
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            40430 non-null  int64 
 1   qid1          40430 non-null  int64 
 2   qid2          40430 non-null  int64 
 3   question1     40430 non-null  object
 4   question2     40430 non-null  object
 5   is_duplicate  40430 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 1.9+ MB
Count


id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

## Data Preprocessing

In [47]:
test_df['q1_processed'] = test_df['question1'].apply(preprocessing)
test_df['q2_processed'] = test_df['question2'].apply(preprocessing)

In [48]:
test_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_processed,q2_processed
0,201359,303345,303346,Why are African-Americans so beautiful?,Why are hispanics so beautiful?,0,why are african americans so beautiful,why are hispanics so beautiful
1,263843,69383,380476,I want to pursue PhD in Computer Science about...,I handle social media for a non-profit. Should...,0,i want to pursue phd in computer science about...,i handle social media for a non profit should...
2,172974,266948,175089,Is there a reason why we should travel alone?,What are some reasons to travel alone?,1,is there a reason why we should travel alone,what are some reasons to travel alone
3,15329,29298,29299,Why are people so obsessed with having a girlf...,How can a single male have a child?,0,why are people so obsessed with having a girlf...,how can a single male have a child
4,209794,314169,314170,What are some good baby girl names starting wi...,What are some good baby girl names starting wi...,0,what are some good baby girl names starting wi...,what are some good baby girl names starting wi...


## Feature Extraction

Basic Features

In [49]:
test_df['q1_no_words'] = test_df['question1'].apply(lambda x : len(x.split(" ")))
test_df['q2_no_words'] = test_df['question2'].apply(lambda x : len(x.split(" ")))
test_df['sum_of_words'] = test_df['q1_no_words'] + df['q2_no_words']

test_df['q1_length'] = test_df['question1'].apply(len)
test_df['q2_length'] = test_df['question2'].apply(len)

test_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_processed,q2_processed,q1_no_words,q2_no_words,sum_of_words,q1_length,q2_length
0,201359,303345,303346,Why are African-Americans so beautiful?,Why are hispanics so beautiful?,0,why are african americans so beautiful,why are hispanics so beautiful,5,5,15.0,39,31
1,263843,69383,380476,I want to pursue PhD in Computer Science about...,I handle social media for a non-profit. Should...,0,i want to pursue phd in computer science about...,i handle social media for a non profit should...,18,25,24.0,106,136
2,172974,266948,175089,Is there a reason why we should travel alone?,What are some reasons to travel alone?,1,is there a reason why we should travel alone,what are some reasons to travel alone,9,7,19.0,45,38
3,15329,29298,29299,Why are people so obsessed with having a girlf...,How can a single male have a child?,0,why are people so obsessed with having a girlf...,how can a single male have a child,9,8,17.0,62,35
4,209794,314169,314170,What are some good baby girl names starting wi...,What are some good baby girl names starting wi...,0,what are some good baby girl names starting wi...,what are some good baby girl names starting wi...,10,12,30.0,51,56


TF-IDF

In [50]:
# TF-IDF:

test_corpus = list(test_df['q1_processed']) + list(test_df['q2_processed'])

test_questions_vectorized = vectorizer.fit_transform(test_corpus)

print("n_samples: %d, n_features: %d" % test_questions_vectorized.shape)
print(test_questions_vectorized.shape)

test_q1_vectors = test_questions_vectorized[:(test_questions_vectorized.shape[0]//2), :]
test_q2_vectors = test_questions_vectorized[(test_questions_vectorized.shape[0]//2):, :]

n_samples: 80860, n_features: 30425
(80860, 30425)


In [51]:
%%time

vect = cosine_similarity(test_q1_vectors[0,:], test_q2_vectors[0,:]).flatten()

for row in range(1, test_q1_vectors.shape[0]):
  value = cosine_similarity(test_q1_vectors[row, :], test_q2_vectors[row, :]).flatten()
  vect = np.hstack((vect, value))

CPU times: user 46.3 s, sys: 151 ms, total: 46.4 s
Wall time: 52.9 s


In [52]:
test_df['cosine_similarity'] = vect
test_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_processed,q2_processed,q1_no_words,q2_no_words,sum_of_words,q1_length,q2_length,cosine_similarity
0,201359,303345,303346,Why are African-Americans so beautiful?,Why are hispanics so beautiful?,0,why are african americans so beautiful,why are hispanics so beautiful,5,5,15.0,39,31,0.436689
1,263843,69383,380476,I want to pursue PhD in Computer Science about...,I handle social media for a non-profit. Should...,0,i want to pursue phd in computer science about...,i handle social media for a non profit should...,18,25,24.0,106,136,0.227562
2,172974,266948,175089,Is there a reason why we should travel alone?,What are some reasons to travel alone?,1,is there a reason why we should travel alone,what are some reasons to travel alone,9,7,19.0,45,38,0.492546
3,15329,29298,29299,Why are people so obsessed with having a girlf...,How can a single male have a child?,0,why are people so obsessed with having a girlf...,how can a single male have a child,9,8,17.0,62,35,0.000000
4,209794,314169,314170,What are some good baby girl names starting wi...,What are some good baby girl names starting wi...,0,what are some good baby girl names starting wi...,what are some good baby girl names starting wi...,10,12,30.0,51,56,0.973193


## Prepping the DataFrame

In [53]:
test_X = test_df.copy()
test_X.drop(["id","qid1","qid2","question1","question2", "is_duplicate", "q1_processed", "q2_processed"], axis = 1, inplace = True)

test_Y = test_df["is_duplicate"]
print(test_X.shape)
print(test_Y.shape)

(40430, 6)
(40430,)


# Random Forest

In [56]:
model = RandomForestClassifier()
model.fit(train_X, train_Y)

RandomForestClassifier()

In [59]:
test_predictions = model.predict(test_X)

# evaluate metrics on test set
lr_test_accuracy = accuracy_score(test_Y, test_predictions)
lr_test_precision = precision_score(test_Y, test_predictions)
lr_test_recall = recall_score(test_Y, test_predictions)
lr_test_f1 = f1_score(test_Y, test_predictions)
lr_test_auc_roc = roc_auc_score(test_Y, test_predictions)

print(f"Logistic Regression Test Accuracy: {lr_test_accuracy * 100:.2f}%")
print(f"Logistic Regression Test Precision: {lr_test_precision * 100:.2f}%")
print(f"Logistic Regression Test Recall: {lr_test_recall * 100:.2f}%")
print(f"Logistic Regression Test F1-Score: {lr_test_f1 * 100:.2f}%")
print(f"Logistic Regression Test AUC-ROC: {lr_test_auc_roc * 100:.2f}%")

Logistic Regression Test Accuracy: 68.52%
Logistic Regression Test Precision: 58.77%
Logistic Regression Test Recall: 48.56%
Logistic Regression Test F1-Score: 53.18%
Logistic Regression Test AUC-ROC: 64.36%
